In [ ]:
from datasets import load_metric, load_dataset

metric = load_metric('glue', 'mrpc')
dataset = load_dataset('glue', 'mrpc')

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [ ]:
!pip install tensorflow-hub sentence-transformers

     |################################| 108 kB 20.5 MB/s 


     |################################| 78 kB 13.0 MB/s 


     |                                | 204 kB 18.6 MB/s eta 0:00:02

     |#                               | 1.1 MB 18.6 MB/s eta 0:00:02

     |###                             | 2.5 MB 18.6 MB/s eta 0:00:02

...

     |################################| 23.3 MB 473 kB/s 


 \

 done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=0cd3f1cf185e981dff662684acafab6315e819e2f38aee29533aa00c9cf02e87
  Stored in directory: /root/.cache/pip/wheels/4e/6f/20/06e0c1e209742a37ce7a5a9aa4e420a3abd5081c65b4b34d0a
Successfully built sentence-transformers


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

torchvision 0.11.1 requires torch==1.10.0, but you'll have torch 1.6.0 which is incompatible.


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
stsb_metric = load_metric('glue', 'stsb')
stsb_dataset = load_dataset('glue', 'stsb')

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [ ]:
import tensorflow_hub as hub 
from sentence_transformers import SentenceTransformer
use_model = hub.load(
    "https://tfhub.dev/google/universal-sentence-encoder/4"
)

distilrobert = SentenceTransformer(
    'stsb-distilroberta-base-v2'
)

INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.


INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 350.00MB


INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 690.00MB


INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB


INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.


In [ ]:
import tensorflow as tf
import math

def use_sts_benchmark(batch):
    sts_encode1 = tf.nn.l2_normalize(
        use_model(tf.constant(batch['sentence1'])), axis=1)
    sts_encode2 = tf.nn.l2_normalize(
        use_model(tf.constant(batch['sentence2'])), axis=1)
    cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
    clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
    scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
    return scores

In [ ]:
def roberta_sts_benchmark(batch):
    sts_encode1 = tf.nn.l2_normalize(
        distilrobert.encode(batch['sentence1']), axis=1)
    sts_encode2 = tf.nn.l2_normalize(
        distilrobert.encode(batch['sentence2']), axis=1)
    cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
    clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
    scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
    return scores

In [ ]:
use_results = use_sts_benchmark(stsb_dataset['validation'])
distilroberta_results = roberta_sts_benchmark(stsb_dataset['validation'])

In [ ]:
references = [item['label'] for item in stsb_dataset['validation']]


In [ ]:
import pandas as pd 

results = {
    "USE": stsb_metric.compute(
        predictions=use_results,
        references=references
    ),
    "DistillRoberta": stsb_metric.compute(
        predictions=distilroberta_results,
        references=references
    )
}

pd.DataFrame(results)

,USE,DistillRoberta
pearson,0.810302,0.888461
spearmanr,0.808917,0.889246
